3-) 

* Edinburgh mutfak eşyaları veri seti1 için hazırlanan ekteki 10 farklı nesneye ait görüntüler 
ile bir nesne sınıflandırma sistemi gerçekleştiriniz. 

* Bunun için nesnelerin RAW olarak isimlendirilmiş klasörlerindeki her bir görüntüyü okuyup 0.25 oranla yeniden 
boyutlandırınız. Bu görüntülerin her birine öncelikle ikinci maddede geliştirdiğiniz ilgi noktası bulucu algoritmayı uygulayıp ilgi noktalarının konumlarını ve ölçeklerini tespit ediniz. 

* Ardından ilgi noktalarını merkeze alan ve noktanın ölçeğine bağlı olarak boyutunu ayarlayacağınız W×W büyüklüğündeki bir pencere içerisinde 4×4 hücre ve her bir hücre için 8 uzunluklu histogramlar belirleyerek yönelimli eğimlerin histogramı (HoG) tanımlayıcısını çalıştırınız. 

* Böylece bir ilgi noktası için 128 uzunluklu bir özellik vektörü oluşturunuz. 

* HoG tanımlayıcısı OpenCV içerisinden hazır olarak kullanılabilir veya HoG’un interpolasyon aşamaları göz ardı edilerek kodlanabilir. 

* Bu şekilde tüm görüntüler için oluşturduğunuz ilgi noktalarını ve tanımlayıcılarını her nesne için IP isimli bir klasör altına görüntülerin dosya ismi ile kaydediniz. 

* İlgi noktalarını belirledikten sonra her nesnenin ilk 8 görüntüsüne ait ilgi noktalarını eğitim son 2 görüntüsüne ait olanları ise test amaçlı kullanarak k en yakın komşu nesne sınıflandırma işlemini gerçekleştiriniz. 

* K en yakın komşu algoritmasında eğitim olmadığından dosyalar içerisinde yer alan özellikler direkt kullanılacaktır. Toplamda 80 eğitim görüntüsü 20 test görüntüsü veride mevcut olduğuna göre test edilecek her bir görüntü için aşağıdaki adımlar uygulanacaktır: 
  - Test görüntüsünün her ilgi noktası için eğitim görüntülerinde SSD ile kendisine en yakın olan 2 komşuyu belirle.
  - Eşlemedeki belirsizlik yeterince düşük ise ve en yakın komşunun ait olduğu sınıf etiketi hangisiyse test görüntüsü için ilgili sınıfa ait olma sayacını 1 arttır. 
  - Not: Alternatif olarak K değeri 3, 5 veya 7 gibi belirlenip komşuların çoğunlukla sahip olduğu etikete göre belirli bir sınıfa ait olma sayacı arttırılabilir. 
  - Tüm ilgi noktaları eşlendikten sonra test görüntüsünün etiketini sayaç değeri en yüksek olan sınıfın etiketi olarak ata.

In [2]:
import os
import cv2
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

def find_interest_points(image, threshold=0.01):
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = np.float32(gray)
    dst = cv2.cornerHarris(gray, 2, 3, threshold)
    dst = cv2.dilate(dst, None)
    return dst

def find_interest_points(image, threshold=0.01):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    corners = cv2.goodFeaturesToTrack(gray, 100, threshold, 10, useHarrisDetector=True)
    return corners.reshape(-1, 2)

def compute_hog(image, interest_points, window_size=128):
    keypoints = [cv2.KeyPoint(x=p[0], y=p[1], size=5) for p in interest_points]
    
    img = cv2.drawKeypoints(image, keypoints, None, color=(0,0,255))
    hog = cv2.HOGDescriptor((window_size, window_size), (16, 16), (8, 8), (8, 8), 9)
    return hog.compute(img)

def knn_classify(X_train, y_train, X_test):
    knn = KNeighborsClassifier(n_neighbors=1)
    knn.fit(X_train, y_train)
    return knn.predict(X_test)


def process_dataset(dataset_folder, new_folder, window_size=64, resize_factor=0.25):
    X = []
    y = []
    for class_id, class_folder in enumerate(os.listdir(dataset_folder)):
        class_folder_path = os.path.join(dataset_folder, class_folder)
        if not os.path.isdir(class_folder_path):
            continue
        for image_file in os.listdir(os.path.join(class_folder_path, 'RAW')):
            image_path = os.path.join(class_folder_path, 'RAW', image_file)
            image = cv2.imread(image_path)
            image_resized = cv2.resize(image, None, fx=resize_factor, fy=resize_factor)

            interest_points = find_interest_points(image_resized)
            hog_features = compute_hog(image_resized, interest_points, window_size)
            
            image_class_folder = os.path.join(new_folder, str(class_folder))
            if not os.path.exists(image_class_folder):
                os.makedirs(image_class_folder)
            np.savetxt(os.path.join(image_class_folder, str(image_file) + ".txt"), hog_features)

            X.append(hog_features)
            y.append(class_id)

def classify_images(dataset_folder):
    X = []
    y = []

    for class_id, class_folder in enumerate(os.listdir(dataset_folder)):
        class_folder_path = os.path.join(dataset_folder, class_folder)
        if not os.path.isdir(class_folder_path):
            continue
        for feat_file in os.listdir(os.path.join(class_folder_path, 'RAW')):
            image_class_folder = os.path.join(dataset_folder, str(class_folder))


            hog_features = np.loadtxt(os.path.join(image_class_folder, str(feat_file) + ".txt"))
            
            X.append(hog_features)
            y.append(class_id)


    X, y = shuffle(X, y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
        
    y_pred = knn_classify(X_train, y_train, X_test)
    accuracy = np.mean(y_pred == y_test)
    print("Accuracy: {:.2f}%".format(accuracy * 100))
    

In [3]:
dataset_folder = 'dataset'
new_folder = 'dataset_processed'
process_dataset(dataset_folder, new_folder, window_size=64, resize_factor=0.25)
classify_images(new_folder)

KeyboardInterrupt: 